In [ ]:
import requests, pickle, os, re
from pathlib import Path
from datetime import date, timedelta, datetime

today=date.today().isoformat()
year=2025
month = (date.today().replace(day=1) - timedelta(days=1)).month#month=7-----------------------------<=Check month!
code_mo="660017"
baseurl=os.getenv('BASE_URL')
api_person = baseurl + "/api/Person"
api_person_list_url = baseurl + "/api/PersonList"
api_police = baseurl + "/api/Polis"
api_usluga = baseurl + "/api/EvnUsluga"
api_usluga_par = baseurl + "/api/EvnUslugaPar"
api_usluga_list = baseurl + "/api/EvnUslugaList"
api_visit = baseurl + "/api/EvnVizitPL"
ecp_searchData = baseurl + "/?c=Search&m=searchData"
ecp_tap_url=baseurl + '/?c=EvnPL&m=loadEvnPLEditForm'
ecp_visit_url=baseurl + '/?c=EvnVizit&m=loadEvnVizitPLEditForm'
ecp_person_url=baseurl + '/?c=Person&m=getPersonEditWindow'
cookie_file=Path("cookies.txt")
os.chdir('C:/Users/User/Anaconda')
today=date.today()
pcr_gepatity_ready=False
pcr_covid_ready=False
fibros_ready=False
cts_ready=False

def month_range(year=datetime.now().year,month=datetime.now().month):
    range_start=date(year,month,1).strftime("%d.%m.%Y")
    range_end=(date(year,month+1,1)-timedelta(days=1)).strftime("%d.%m.%Y")
    return range_start+" - " + range_end

def login():
    print("Logining")
    headers = {
            "Login": "GIBBot1",
            "Password": "Qwerty1!"
    }
    response=session.request("GET", f"{baseurl}/api/user/login", headers=headers).json()
    print(response)

def ecp_search_person(name='', patronymic = '', surname='', birthday='', police='', snils=''):
    if snils!='':
        if type(snils)==str:
            list_snils = re.findall(r'\d+', snils)
            snils = ''.join(list_snils)
        if type(snils)==int:
            snils="{:011d}".format(snils)
        snils=snils[:3]+"-"+snils[3:6]+"-"+snils[6:9]+"-"+snils[9:]
    params={
        "PersonFirName_FirName": name,
        "PersonSecName_SecName": patronymic,
        "PersonSurName_SurName": surname,
        "PersonBirthDay_BirthDay": birthday,
        "Polis_Num": police,
        "Person_Snils": snils,
    }
    return session.request("POST", baseurl + '/?c=Person&m=getPersonSearchGrid', params=params).json()

date_range=month_range(year,month)
session = requests.session()
login()

Loading cookies
Cookies found!
Logining
{'error_code': 0, 'sess_id': '6p1i7b566t35gbi71fd4sp0ct4'}


In [ ]:
from openpyxl import load_workbook
import os
def snils_to_prikreplenie(snils):
    ecp_search_person_resp=ecp_search_person(snils=snils)
    if len(ecp_search_person_resp['data'])==0:
        return 'no pacients found ' + str(len(ecp_search_person_resp['data']))
    if len(ecp_search_person_resp['data'])>1:
        return 'many pacients found ' + str(len(ecp_search_person_resp['data']))
    return ecp_search_person_resp['data'][0]['Lpu_Nick']

def process_excel_file(file_path, process_func):
    """
    Process Excel file to add 'Прикрепление' column based on 'СНИЛС' values.
    
    Args:
        file_path: Path to the Excel file
        process_func: Function that converts SNILS to Prikreplenie value
    """
    # First, let's find the header row and the СНИЛС column
    wb = load_workbook(file_path)
    ws = wb.active
    
    header_row = None
    snils_col = None
    
    # Search for the header row containing 'СНИЛС'
    for row_idx, row in enumerate(ws.iter_rows(values_only=True), 1):
        if 'СНИЛС' in row:
            header_row = row_idx
            snils_col = row.index('СНИЛС') + 1  # Convert to 1-based index for openpyxl
            break
    
    if header_row is None or snils_col is None:
        raise ValueError("Could not find 'СНИЛС' column in the Excel file")
    
    # Find the last column to add our new column
    last_col = ws.max_column + 1
    
    # Add the header for the new column
    ws.cell(row=header_row, column=last_col, value='Прикрепление')
    
    # Process each row below the header
    for row_idx in range(header_row + 1, ws.max_row + 1):
        snils_value = ws.cell(row=row_idx, column=snils_col).value
        
        # Skip empty cells
        if snils_value:
            print('Apply the transformation function to snils', snils_value,end='. ')
            
            prikreplenie_value = process_func(str(snils_value))
            
            print('Write the result to the new column:',prikreplenie_value)
            ws.cell(row=row_idx, column=last_col, value=prikreplenie_value)
    
    # Create a temporary file name
    temp_file = file_path + ".temp"
    
    # Save to temporary file
    wb.save(temp_file)
    wb.close()
    
    # Replace original file with the new one
    os.replace(temp_file, file_path)
    
    print(f"Successfully processed file: {file_path}")

process_excel_file(r"C:\Users\User\Anaconda\2025-09\ГИБ HCV ПЦР сентябрь всё.xlsx", snils_to_prikreplenie)

ValueError: Could not find 'СНИЛС' column in the Excel file

In [14]:

name=''
patronymic = ''
surname=''
birthday=''
police=''
snils=11966934300
if snils!='':
    if type(snils)==str:
        list_snils = re.findall(r'\d+', snils)
        snils = ''.join(list_snils)
    if type(snils)==int:
        snils="{:011d}".format(snils)
    snils=snils[:3]+"-"+snils[3:6]+"-"+snils[6:9]+"-"+snils[9:]
params={
    "PersonFirName_FirName": name,
    "PersonSecName_SecName": patronymic,
    "PersonSurName_SurName": surname,
    "PersonBirthDay_BirthDay": birthday,
    "Polis_Num": police,
    "Person_Snils": snils,
}
ecp_search_person_resp=session.request("POST", baseurl + '/?c=Person&m=getPersonSearchGrid', data=params)

In [15]:
ecp_search_person_resp.request.url

'https://ecp.mis66.ru/?c=Person&m=getPersonSearchGrid'

In [18]:
ecp_search_person_resp.json()

{'data': [{'Person_id': '660910001134339',
   'Server_id': '13003640',
   'PersonEvn_id': '82547418',
   'Person_IsInErz': '2',
   'Person_Phone': '9920106275',
   'Person_Inn': None,
   'Person_IsUnknownSign': None,
   'Person_IsUnknown': None,
   'FamilyStatus_id': None,
   'SocStatus_id': '10000072',
   'SocStatusFed_id': '8',
   'SocStatusFed_Code': '8',
   'PersonCard_id': None,
   'PersonCard_Lpu_id': None,
   'PersonCard_LpuRegions': None,
   'Person_isOftenCaller': '1',
   'Person_IsRefuse': 'false',
   'Person_IsDead': 'false',
   'existsProjectDeathSvid': '1',
   'existsProjectPntDeathSvid': '1',
   'Person_IsFedLgot': 'true',
   'Person_IsRegLgot': 'false',
   'Person_Lgots': 'true',
   'Person_Is7Noz': 'false',
   'UAddress_AddressText': '620109, РОССИЯ, СВЕРДЛОВСКАЯ ОБЛ, Г ЕКАТЕРИНБУРГ, РЕПИНА УЛ, д. 21, кв. ОБЩ',
   'PAddress_AddressText': '620109, РОССИЯ, СВЕРДЛОВСКАЯ ОБЛ, Г ЕКАТЕРИНБУРГ, РЕПИНА УЛ, д. 21, кв. 44',
   'Person_IsBDZ': 'false',
   'PersonQuarantine_IsOn': 

In [9]:
delimeter=";"
for snils in snils_list:
    ecp_search_person_resp=ecp_search_person(snils=snils)
    if len(ecp_search_person_resp['data'])==0:
        print('no pacients found ' + str(len(ecp_search_person_resp['data'])))
        continue
    if len(ecp_search_person_resp['data'])>1:
        print('many pacients found ' + str(len(ecp_search_person_resp['data'])))
        continue
    if len(ecp_search_person_resp['data'])==1:
        print(ecp_search_person_resp['data'][0]['Person_Snils'],
              str(ecp_search_person_resp['data'][0]['Polis_Num']),
              str(ecp_search_person_resp['data'][0]['Person_IsDead']),
              ecp_search_person_resp['data'][0]['PersonSurName_SurName'],
              ecp_search_person_resp['data'][0]['PersonFirName_FirName'],
              ecp_search_person_resp['data'][0]['PersonSecName_SecName'],
              str(ecp_search_person_resp['data'][0]['PersonBirthDay_BirthDay']),
              ecp_search_person_resp['data'][0]['Lpu_Nick'],
              sep=";",
        )

05983962426;6648320846001177;false;КИЧИГИН;ПАВЕЛ;АЛЕКСАНДРОВИЧ;03.11.1976;ГАУЗ СО "ГП № 3 г. Нижний Тагил"
06492527281;6648420834000320;false;ВОРОБЬЕВ;ВИТАЛИЙ;МИХАЙЛОВИЧ;15.11.1975;ГАУЗ СО "Серовская ГБ"
09590550600;6652210848002260;false;НОВОСЕЛОВ;ВАСИЛИЙ;ОЛЕГОВИЧ;01.07.1987;None
11251084200;6653510896000200;false;ПРИМАК;ЛЮДМИЛА;ПЕТРОВНА;03.06.1984;None
12797523094;6656130839001665;false;СИРОТИН;РОДИОН;СЕРГЕЕВИЧ;10.03.1968;ГБУЗ СО "ЦГКБ №1"
12830972564;6652630885000689;false;КАРЕЛЬЦЕВА;ОЛЬГА;БОРИСОВНА;14.07.1963;ГАУЗ СО "ЦГКБ № 3"
13276386369;6655510843000798;false;УПОРОВ;АНАТОЛИЙ;ИГНАТЬЕВИЧ;06.04.1984;ГАУЗ СО "Талицкая ЦРБ"
14153158835;6656110820001031;false;СМИРНОВ;АНДРЕЙ;ВЛАДИМИРОВИЧ;29.03.1988;ГАУЗ СО "Ревдинская ГБ"
15813447774;6650800897001643;false;ТОКАРЕВА;АЛЕНА;ОЛЕГОВНА;02.09.1991;ФБУЗ МСЧ 32 ФМБА России Заречный
no pacients found 0
00803365212;6658430836000209;false;НАЙМУШИН;АЛЕКСАНДР;ЮРЬЕВИЧ;13.01.1965;ГАУЗ СО "Режевская ЦРБ"
00804142399;6651120872000588;false;АФАНАСЬЕВА;ЕВ

In [5]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import pandas as pd
import openpyxl
import os
import csv
from openpyxl.utils import get_column_letter
import threading

class Application(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Фильтрация организаций")
        self.geometry("600x400")
        
        # Стиль
        self.style = ttk.Style()
        self.style.theme_use('clam')
        
        # Переменные
        self.input_file = tk.StringVar()
        self.output_folder = tk.StringVar()
        self.column_name = tk.StringVar()
        self.sheet_name = tk.StringVar()
        self.head_line = tk.IntVar()
        self.prefix = tk.StringVar()
        self.status_text = tk.StringVar()
        self.progress_value = tk.DoubleVar()
        self.columns_list = []
        self.sheet_names = []
        
        self.create_widgets()
        self.protocol("WM_DELETE_WINDOW", self.on_close)
    
    def create_widgets(self):
        # Фрейм для ввода данных
        input_frame = ttk.LabelFrame(self, text="Параметры обработки")
        input_frame.pack(padx=10, pady=5, fill='x')

        # Выбор файла
        ttk.Label(input_frame, text="Входной файл:").grid(row=0, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.input_file, width=40).grid(row=0, column=1, padx=5, pady=2)
        ttk.Button(input_frame, text="Обзор...", command=self.browse_file).grid(row=0, column=2, padx=5, pady=2)

        # Выбор листа (только для Excel)
        self.sheet_label = ttk.Label(input_frame, text="Лист в книге:")
        self.sheet_label.grid(row=1, column=0, sticky='w', padx=5, pady=2)
        self.sheet_combobox = ttk.Combobox(
            input_frame, 
            textvariable=self.sheet_name, 
            state="readonly",
            width=20
        )
        self.sheet_combobox.grid(row=1, column=1, sticky='w', padx=5, pady=2)
        
        # Выбор столбца
        ttk.Label(input_frame, text="Столбец с организациями:").grid(row=2, column=0, sticky='w', padx=5, pady=2)
        self.column_combobox = ttk.Combobox(
            input_frame, 
            textvariable=self.column_name, 
            state="readonly",
            width=20
        )
        self.column_combobox.grid(row=2, column=1, sticky='w', padx=5, pady=2)
        
        # Выбор папки для сохранения
        ttk.Label(input_frame, text="Папка для результатов:").grid(row=3, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.output_folder, width=40).grid(row=3, column=1, padx=5, pady=2)
        ttk.Button(input_frame, text="Обзор...", command=self.browse_output).grid(row=3, column=2, padx=5, pady=2)
        
                # head_line
        ttk.Label(input_frame, text="Номер строки с шапкой:").grid(row=4, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.head_line, width=40).grid(row=4, column=1, padx=5, pady=2)
        self.head_line.set(1)
        
        # prefix
        ttk.Label(input_frame, text="Префикс для названия файлов:").grid(row=5, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.prefix, width=40).grid(row=5, column=1, padx=5, pady=2)
        self.prefix.set('Гепатиты 244-п февраль')
        
        # Прогресс-бар
        progress_frame = ttk.Frame(self)
        progress_frame.pack(padx=10, pady=10, fill='x')
        self.progress = ttk.Progressbar(progress_frame, variable=self.progress_value, maximum=100)
        self.progress.pack(fill='x')
        
        # Статус
        status_frame = ttk.Frame(self)
        status_frame.pack(padx=10, pady=5, fill='x')
        ttk.Label(status_frame, textvariable=self.status_text).pack(side='left')
        
        # Кнопки управления
        button_frame = ttk.Frame(self)
        button_frame.pack(padx=10, pady=10)
        ttk.Button(button_frame, text="Старт", command=self.start_processing).pack(side='left', padx=5)
        ttk.Button(button_frame, text="Выход", command=self.on_close).pack(side='left', padx=5)

    def browse_file(self):
        filetypes = (
            ('Excel files', '*.xlsx *.xls'),
            ('CSV files', '*.csv'),
            ('All files', '*.*')
        )
        filename = filedialog.askopenfilename(filetypes=filetypes)
        if filename:
            self.input_file.set(filename)
            if filename.lower().endswith(('.xls', '.xlsx')):
                self.load_sheets(filename)
            else:
                self.load_columns(filename)

    def load_sheets(self, filename):
        try:
            wb = openpyxl.load_workbook(filename, read_only=True)
            self.sheet_names = wb.sheetnames
            wb.close()
            self.sheet_combobox['values'] = self.sheet_names
            if self.sheet_names:
                self.sheet_name.set(self.sheet_names[0])
                self.load_columns(filename, self.sheet_names[0])
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось прочитать листы: {str(e)}")

    def load_columns(self, filename, sheet_name=None):
        try:
            if filename.endswith(('.xls', '.xlsx')):
                df = pd.read_excel(filename, sheet_name=sheet_name, nrows=0)
            else:
                df = pd.read_csv(filename, nrows=0, encoding='utf-8')
            self.columns_list = df.columns.tolist()
            self.column_combobox['values'] = self.columns_list
            if self.columns_list:
                self.column_name.set(self.columns_list[0])
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось прочитать заголовки: {str(e)}")
            self.column_combobox['values'] = []
            self.column_name.set('')
            
    def browse_output(self):
        folder = filedialog.askdirectory()
        if folder:
            self.output_folder.set(folder)

    def start_processing(self):
        if not self.validate_input():
            return
        
        # Запуск в отдельном потоке
        thread = threading.Thread(target=self.process_data)
        thread.start()

    def validate_input(self):
        if not self.input_file.get():
            messagebox.showwarning("Ошибка", "Выберите входной файл!")
            return False
        if self.input_file.get().endswith(('.xls', '.xlsx')) and not self.sheet_name.get():
            messagebox.showwarning("Ошибка", "Выберите лист в книге!")
            return False
        if not self.column_name.get():
            messagebox.showwarning("Ошибка", "Выберите столбец с организациями!")
            return False
        return True

    def process_data(self):
        try:
            self.update_status("Начало обработки...")
            input_file = self.input_file.get()
            column_name = self.column_name.get()
            output_folder = self.output_folder.get() or 'output'
            sheet = self.sheet_name.get() if self.sheet_name.get() else None
            
            # Загрузка данных и информации о ширине столбцов
            if input_file.endswith('.csv'):
                df = pd.read_csv(input_file)
                col_widths = None
            else:
                wb = openpyxl.load_workbook(input_file)
                ws = wb[sheet] if sheet else wb.active
                col_widths = []
                for col in range(1, ws.max_column + 1):
                    col_letter = get_column_letter(col)
                    col_widths.append(ws.column_dimensions[col_letter].width if col_letter in ws.column_dimensions else 8.43)
                df = pd.read_excel(input_file)
                wb.close()
            
            organizations = df[column_name].unique()
            total = len(organizations)
            os.makedirs(output_folder, exist_ok=True)
            
            for i, org in enumerate(organizations, 1):
                org_data = df[df[column_name] == org]
                clean_org_name = "".join([c if c.isalnum() or c in (' ', '_') else '_' for c in str(org)])
                output_path = os.path.join(output_folder, f"{clean_org_name}.xlsx")
                
                with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
                    org_data.to_excel(writer, index=False, sheet_name='Data')
                    if col_widths is not None:
                        worksheet = writer.sheets['Data']
                        for idx, width in enumerate(col_widths, 1):
                            worksheet.column_dimensions[get_column_letter(idx)].width = width
                
                self.progress_value.set((i / total) * 100)
                self.update_status(f"Обработано: {i} из {total} ({org})")
            
            self.update_status("Обработка завершена успешно!")
            messagebox.showinfo("Готово", f"Создано {total} файлов в папке:\n{output_folder}")
        
        except Exception as e:
            messagebox.showerror("Ошибка", str(e))
            self.update_status("Ошибка обработки!")
        finally:
            self.progress_value.set(0)
    
    def update_status(self, message):
        self.status_text.set(message)
        self.update_idletasks()
    
    def on_close(self):
        if messagebox.askokcancel("Выход", "Вы уверены, что хотите выйти?"):
            self.destroy()
# def filter_and_save_organizations(input_file, column_name, output_folder='output', sheet='Лист1', head_line=1,prefix=''):
#     """
#     Функция для фильтрации данных по организациям и сохранения в отдельные файлы
    
#     Параметры:
#     input_file (str): Путь к входному файлу (CSV или Excel)
#     column_name (str): Название столбца с организациями
#     output_folder (str): Папка для сохранения результатов
#     """
    
#     # Загрузка данных
#     if input_file.endswith('.csv'):
#         df = pd.read_csv(input_file)
#     elif input_file.endswith(('.xls', '.xlsx')):
#         wb = openpyxl.load_workbook(input_file)
#         ws = wb.active
#         col_widths=[]
#         for col in range(1,ws.max_column+1):
#             col_letter = get_column_letter(col)
#             if col_letter in ws.column_dimensions:
#                 col_widths.append(ws.column_dimensions[col_letter].width)
#             else:
#                 col_widths.append(8.43)
        
#         # col_widths = [ws.column_dimensions[col].width for col in ws.column_dimensions]
#         # print(len(col_widths),col_widths)
#         df = pd.read_excel(input_file, sheet_name=sheet, header=head_line-1)
#         wb.close()
#     else:
#         raise ValueError("Формат файла не поддерживается. Используйте CSV или Excel")
#     print(df)
#     # Создаем папку для результатов, если ее нет
#     os.makedirs(output_folder, exist_ok=True)
    
#     # Получаем уникальные организации
#     organizations = df[column_name].unique()

#     # Функция для очистки имени файла
#     def clean_filename(name):
#         # Заменяем недопустимые символы в имени файла
#         return "".join([c if c.isalnum() or c in (' ', '_') else '' for c in str(name)])
    
#     # Сохраняем данные по каждой организации
#     for org in organizations:
#         # Фильтруем данные
#         org_data = df[df[column_name] == org]
        
#         # Очищаем имя организации для имени файла
#         clean_org_name = clean_filename(org)

#         # Формируем путь к файлу
#         if org in vipnets:
#             output_path = os.path.join(output_folder, prefix+' '+clean_org_name+vipnets[org]+'.xlsx')
#         else:
#             print('нет правила для организации:', org)
#             output_path = os.path.join(output_folder, f"{clean_org_name}.xlsx")
        
#         # Сохраняем в xlsx
#         with pd.ExcelWriter(output_path, engine='openpyxl') as writer: 
#             # org_data.to_excel(output_path, index=False)
#             org_data.to_excel(writer, index=False, sheet_name='Data')
#             if col_widths is not None:
#                 worksheet = writer.sheets['Data']
#                 for idx, width in enumerate(col_widths, 1):
#                     column_letter = openpyxl.utils.get_column_letter(idx)
#                     worksheet.column_dimensions[column_letter].width = width
    
#     print(f"Создано {len(organizations)} файлов в папке '{output_folder}'")

if __name__ == "__main__":
    # Настройки
    # input_file = 'гепатиты 244-п февраль.xlsx'  # Укажите путь к вашему файлу
    # column_name = 'Прикрепление'  # Укажите название столбца с организациями
    # output_folder = '244'  # Название папки для результатов
    # sheet = 'Результат'#'asulon+'#'region+'
    # head_line=9
    # prefix='Гепатиты 244-п февраль'
    
    vipnets = {}
    with open('act_addr.txt') as in_file:
        csv_reader = csv.reader(in_file)
        for row in csv_reader:
            vipnets[row[0]]=row[1]
    
    # Запуск обработки
    # filter_and_save_organizations(
    #     input_file=input_file,
    #     column_name=column_name,
    #     output_folder=output_folder,
    #     sheet=sheet,
    #     head_line=head_line,
    #     prefix=prefix
    # )
    app = Application()
    app.mainloop()

In [ ]:
#------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------
#----------------------------ИСПОЛЬЗУЙ НИЖНИЙ БЛОКНОТ--------------------------------------------------------
#------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------


In [5]:
import os
import csv
import threading
import openpyxl
from openpyxl.utils import get_column_letter
import pandas as pd
import tkinter as tk
from tkinter import ttk, filedialog, messagebox, simpledialog

class Application(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Фильтрация организаций")
        self.geometry("750x500")
        
        # self.style = ttk.Style()
        # self.style.theme_use('clam')
        
        # Переменные
        self.input_file = tk.StringVar()
        self.output_folder = tk.StringVar()
        self.column_name = tk.StringVar()
        self.sheet_name = tk.StringVar()
        self.header_row = tk.IntVar(value=1)
        self.status_text = tk.StringVar()
        self.progress_value = tk.DoubleVar()
        self.prefix = tk.StringVar()
        self.columns_list = []
        self.sheet_names = []
        self.preview_data = []

        # Флаг для остановки потока
        self.running = False
        
        self.create_widgets()
        self.protocol("WM_DELETE_WINDOW", self.on_close)
    
    def create_widgets(self):
        input_frame = ttk.LabelFrame(self, text="Параметры обработки")
        input_frame.pack(padx=10, pady=5, fill='x')
        
        # Выбор файла
        ttk.Label(input_frame, text="Входной файл:").grid(row=0, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.input_file, width=40).grid(row=0, column=1, padx=5, pady=2)
        ttk.Button(input_frame, text="Обзор...", command=self.browse_file).grid(row=0, column=2, padx=5, pady=2)
        
        # Выбор листа
        self.sheet_label = ttk.Label(input_frame, text="Лист в книге:")
        self.sheet_label.grid(row=1, column=0, sticky='w', padx=5, pady=2)
        self.sheet_combobox = ttk.Combobox(input_frame, textvariable=self.sheet_name, state="readonly", width=20)
        self.sheet_combobox.grid(row=1, column=1, sticky='w', padx=5, pady=2)
        
        # Выбор строки заголовка
        ttk.Label(input_frame, text="Строка с заголовками:").grid(row=2, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.header_row, width=5).grid(row=2, column=1, sticky='w', padx=5, pady=2)
        ttk.Button(input_frame, text="Предпросмотр", command=self.show_preview).grid(row=2, column=2, padx=5, pady=2)
        
        # Выбор столбца
        ttk.Label(input_frame, text="Столбец с организациями:").grid(row=3, column=0, sticky='w', padx=5, pady=2)
        self.column_combobox = ttk.Combobox(input_frame, textvariable=self.column_name, state="readonly", width=20)
        self.column_combobox.grid(row=3, column=1, sticky='w', padx=5, pady=2)
        
        # Выбор папки
        ttk.Label(input_frame, text="Папка для результатов:").grid(row=4, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.output_folder, width=40).grid(row=4, column=1, padx=5, pady=2)
        ttk.Button(input_frame, text="Обзор...", command=self.browse_output).grid(row=4, column=2, padx=5, pady=2)

        # prefix
        ttk.Label(input_frame, text="Префикс для названия файлов:").grid(row=5, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.prefix, width=40).grid(row=5, column=1, padx=5, pady=2)
        self.prefix.set('Гепатиты 244-п февраль')
        
        # Прогресс и статус
        progress_frame = ttk.Frame(self)
        progress_frame.pack(padx=10, pady=10, fill='x')
        self.progress = ttk.Progressbar(progress_frame, variable=self.progress_value, maximum=100)
        self.progress.pack(fill='x')
        
        status_frame = ttk.Frame(self)
        status_frame.pack(padx=10, pady=5, fill='x')
        ttk.Label(status_frame, textvariable=self.status_text).pack(side='left')
        
        # Кнопки
        button_frame = ttk.Frame(self)
        button_frame.pack(padx=10, pady=10)
        ttk.Button(button_frame, text="Старт", command=self.start_processing).pack(side='left', padx=5)
        ttk.Button(button_frame, text="Выход", command=self.on_close).pack(side='left', padx=5)
        
    def browse_file(self):
        filetypes = (("Excel/CSV файлы", "*.xlsx *.xls *.csv"), ("Все файлы", "*.*"))
        filename = filedialog.askopenfilename(filetypes=filetypes)
        if filename:
            self.input_file.set(filename)
            self.load_file_metadata(filename)
    
    def load_file_metadata(self, filename):
        try:
            if filename.lower().endswith(('.xls', '.xlsx')):
                self.load_sheets(filename)
                self.load_columns()
            else:
                self.load_columns()
            
            self.show_preview()
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Ошибка загрузки файла: {str(e)}")
    
    def load_sheets(self, filename):
        try:
            wb = openpyxl.load_workbook(filename, read_only=True)
            self.sheet_names = wb.sheetnames
            wb.close()
            self.sheet_combobox['values'] = self.sheet_names
            self.sheet_name.set(self.sheet_names[0])
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось прочитать листы: {str(e)}")
    
    def load_columns(self):
        try:
            header_row = self.header_row.get() - 1  # Конвертируем в 0-based индекс
            filename = self.input_file.get()
            
            if filename.endswith(('.xls', '.xlsx')):
                df = pd.read_excel(
                    filename,
                    sheet_name=self.sheet_name.get(),
                    header=header_row,
                    nrows=0
                )
            else:
                df = pd.read_csv(
                    filename,
                    header=header_row,
                    nrows=0,
                    encoding='utf-8'
                )
            
            self.columns_list = df.columns.tolist()
            self.column_combobox['values'] = self.columns_list
            if self.columns_list:
                self.column_name.set(self.columns_list[0])
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось прочитать заголовки: {str(e)}")
    
    def show_preview(self):
        try:
            filename = self.input_file.get()
            if not filename:
                return
            
            # Чтение первых 15 строк для предпросмотра
            nrows = 15
            header_row = self.header_row.get() - 1
            
            if filename.endswith(('.xls', '.xlsx')):
                df = pd.read_excel(
                    filename,
                    sheet_name=self.sheet_name.get(),
                    header=None,
                    nrows=nrows
                )
            else:
                df = pd.read_csv(
                    filename,
                    header=None,
                    nrows=nrows,
                    encoding='utf-8'
                )
            
            # Создание окна предпросмотра
            preview_window = tk.Toplevel(self)
            preview_window.title("Предпросмотр данных")
            
            # Создаем Treeview с прокруткой
            frame = ttk.Frame(preview_window)
            frame.pack(expand=True, fill='both', padx=5, pady=5)
            
            tree = ttk.Treeview(frame, columns=list(range(len(df.columns))), show="headings")
            vsb = ttk.Scrollbar(frame, orient="vertical", command=tree.yview)
            hsb = ttk.Scrollbar(frame, orient="horizontal", command=tree.xview)
            tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)
            
            tree.grid(row=0, column=0, sticky='nsew')
            vsb.grid(row=0, column=1, sticky='ns')
            hsb.grid(row=1, column=0, sticky='ew')
            frame.grid_columnconfigure(0, weight=1)
            frame.grid_rowconfigure(0, weight=1)
            
            # Настройка столбцов
            for i in range(len(df.columns)):
                tree.heading(i, text=f"Col {i+1}")
                tree.column(i, width=100, anchor='w')
            
            # Вставка данных
            for index, row in df.iterrows():
                tree.insert("", "end", values=tuple(row))
            
            # Подсветка строки с заголовками
            if header_row < len(df):
                tree.tag_configure('header', background='#b3e6ff')
                children = tree.get_children()
                if header_row < len(children):
                    tree.item(children[header_row], tags=('header',))
            
            # Обработка двойного клика
            def on_double_click(event):
                # Получаем выбранную строку и столбец
                item = tree.identify_row(event.y)
                column = tree.identify_column(event.x)
                
                if item and column:
                    # Получаем индекс строки
                    children = tree.get_children()
                    row_index = children.index(item) + 1  # +1 так как нумерация с 1
                    
                    # Получаем индекс столбца
                    col_index = int(column.replace("#", "")) - 1  # Преобразуем "#1" в 0
                    
                    # Устанавливаем новую строку заголовков
                    self.header_row.set(row_index)
                    self.load_columns()
                    
                    # Устанавливаем выбранный столбец
                    if self.columns_list and col_index < len(self.columns_list):
                        self.column_name.set(self.columns_list[col_index])
                    
                    # Закрываем окно предпросмотра
                    preview_window.destroy()
            
            # Привязываем обработчик двойного клика
            tree.bind("<Double-1>", on_double_click)
            
            # Кнопка закрытия
            button_frame = ttk.Frame(preview_window)
            button_frame.pack(fill='x', padx=5, pady=5)
            ttk.Button(button_frame, text="Закрыть", command=preview_window.destroy).pack(side='right')
            
            # Установка размера окна
            preview_window.geometry("800x400")
            
            # Инструкция для пользователя
            instruction = ttk.Label(
                preview_window,
                text="Дважды щелкните по ячейке, чтобы выбрать строку заголовков и столбец для фильтрации",
                font=('Arial', 10)
            )
            instruction.pack(pady=5)
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Ошибка предпросмотра: {str(e)}")
    
    def browse_output(self):
        folder = filedialog.askdirectory()
        if folder:
            self.output_folder.set(folder)
    
    def start_processing(self):
        if not self.validate_input():
            return
        # Запуск обработки в отдельном потоке
        self.running = True
        self.processing_thread = threading.Thread(target=self.process_data)
        self.processing_thread.start()
    
    def validate_input(self):
        if not self.input_file.get():
            messagebox.showwarning("Ошибка", "Выберите входной файл!")
            return False
        if self.input_file.get().endswith(('.xls', '.xlsx')) and not self.sheet_name.get():
            messagebox.showwarning("Ошибка", "Выберите лист в книге!")
            return False
        if not self.column_name.get():
            messagebox.showwarning("Ошибка", "Выберите столбец с организациями!")
            return False
        return True
    
    def process_data(self):
        vipnets = {}
        with open('act_addr.txt') as in_file:
            csv_reader = csv.reader(in_file)
            for row in csv_reader:
                vipnets[row[0]]=row[1]
        try:
            self.after(0, self.update_status, "Начало обработки...")
            
            input_file = self.input_file.get()
            column_name = self.column_name.get()
            output_folder = self.output_folder.get() or 'output'
            sheet = self.sheet_name.get() if self.sheet_name.get() else None
            header_row = self.header_row.get() - 1
            
            # Загрузка данных
            if input_file.endswith('.csv'):
                df = pd.read_csv(input_file, header=header_row)
                col_widths = None
            else:
                wb = openpyxl.load_workbook(input_file)
                ws = wb[sheet] if sheet else wb.active
                
                # Чтение ширины столбцов
                col_widths = []
                for col in range(1, ws.max_column + 1):
                    col_letter = get_column_letter(col)
                    col_widths.append(
                        ws.column_dimensions[col_letter].width 
                        if col_letter in ws.column_dimensions 
                        else 8.43
                    )
                
                df = pd.read_excel(input_file, sheet_name=sheet, header=header_row)
                wb.close()

            # Обработка данных
            organizations = df[column_name].unique()
            total = len(organizations)
            os.makedirs(output_folder, exist_ok=True)
            
            for i, org in enumerate(organizations, 1):
                if not self.running:  # Проверка флага
                    break
                org_data = df[df[column_name] == org]
                clean_org_name = "".join([c if c.isalnum() or c in (' ', '_') else '' for c in str(org)])
                if org in vipnets:
                    output_path = os.path.join(output_folder, self.prefix.get()+' '+clean_org_name+vipnets[org]+'.xlsx')
                else:
                    print('нет правила для организации:', org)
                    output_path = os.path.join(output_folder, f"{clean_org_name}.xlsx")
                
                with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
                    org_data.to_excel(writer, index=False, sheet_name='Data')
                    if col_widths is not None:
                        worksheet = writer.sheets['Data']
                        for idx, width in enumerate(col_widths, 1):
                            worksheet.column_dimensions[get_column_letter(idx)].width = width
                progress = (i / total) * 100
                self.after(0, self.update_progress, progress)
                self.after(0, self.update_status, f"Обработано: {i} из {total} ({org})")
            
            if self.running:
                self.after(0, self.update_status, "Обработка завершена успешно!")
                self.after(0, lambda: messagebox.showinfo(
                    "Готово", 
                    f"Создано {total} файлов в папке:\n{output_folder}"
                ))
        
        except Exception as e:
            self.after(0, lambda: messagebox.showerror("Ошибка", str(e)))
            self.after(0, self.update_status, "Ошибка обработки!")
        finally:
            self.after(0, self.reset_progress)
            self.running = False
    
    def update_progress(self, value):
        self.progress_value.set(value)
    
    def reset_progress(self):
        self.progress_value.set(0)
    
    def update_status(self, message):
        self.status_text.set(message)
    
    def on_close(self):
        if messagebox.askokcancel("Выход", "Вы уверены, что хотите выйти?"):
            self.running = False  # Остановка фонового потока
            if hasattr(self, 'processing_thread') and self.processing_thread.is_alive():
                self.processing_thread.join(timeout=1)  # Ожидание завершения потока
            self.destroy()

if __name__ == "__main__":
    app = Application()
    app.mainloop()

нет правила для организации: ГАУЗ СО ГИБ г. Н.Тагил


#------------------------------------>>>4. PCR COVID-19<<<-------------------------------------
if not pcr_covid_ready:
    pcr_covid_file="S:\\14 - Лаборатория\\списки обследованных на cov19\\обследованные на cov19 2024.ods"
    pcr_covid=pd.read_excel(pcr_covid_file, sheet_name='Данные')
    pcr_covid.dropna()
    pcr_covid_selected = pcr_covid.loc[(pcr_covid['Название организации-заказчика'] != 'ГАУЗ СО ГИБ г.Н.Тагил')&(pcr_covid['Дата отчетности'].dt.month == month)]
    pcr_covid_ready = True
    
for index, row in pcr_covid_selected.iterrows():
    zap_count+=1
    usl_count+=1
    date=row['Дата отчетности'].date().isoformat()
    npolis=('%f'%row['ОМС']).rstrip('0').rstrip('.')
    code_npr_mo,code_npr_doctor='660010', '38945'
    if row['Название организации-заказчика']=='ГАУЗ СО ГБ №4':
        code_npr_mo,code_npr_doctor='660013', '7654'
    history_num=(date+npolis)[:24]
    if type(row['Отчество']) == str:
        patronymic=row['Отчество'].strip()
    else:
        patronymic=row['Отчество'] = ''
    zap, per = generate_zap_per_app(zap_count, 
                                    usl_count, 
                                    history_num=history_num,
                                    date=date,
                                    npr_mo=code_npr_mo,
                                    npr_doctor=code_npr_doctor,
                                    podr='90000', 
                                    profil='34',
                                    prvs='26',
                                    iddokt='7229',
                                    ds='J06.9',
                                    code_usl='A26.08.008.102', 
                                    npolis=npolis,
                                    fam=row['Фамилия'].strip(),
                                    im=row['Имя'].strip(),
                                    ot=patronymic,
                                    w=sex_is_magic(patronymic),
                                    dr=row['Дата рождения'].date().isoformat(),
                                    snils=parse_snils(row['СНИЛС']),
                                    vpolis=3, 
                                    smo="66004", 
                                    p_cel=12,
                                    razdel_usl=301,
                                    kol_usl=1,
                                    usl_ok=3,
                                    vidpom=13,
                                    npr_num=history_num,
                                    npr_type=4,
                                    idsp=28,
                                    rslt=314,
                                    for_pom=3,
                                    okato="65476000000",
                                    mes=1101
                                   )
    pers.append(per)
    zaps.append(zap)
    print("added pcr-grip:", zap, per)
print(datetime.now())